In [1]:
#导入需要的各种包
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from spectral import *
import os

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
#读取数据集
data = np.array([])
target = np.array([])
n=0
for i in os.listdir('9个类别的数据集-train/'):
    a,b=os.path.splitext(i)
    new=sio.loadmat('9个类别的数据集-train/'+i)[a]
    for j in range(new.shape[0]):
        target=np.append(target,n)
    data = np.append(data,new)
    n=n+1
data = np.reshape(data,(-1,200))
target = np.reshape(target,(-1,1))

In [3]:
#划分数据集
xtrain,xdev,ytrain,ydev=train_test_split(data,target,test_size=0.2)

In [4]:
#数据预处理 归一化
scaler=StandardScaler()
scaler.fit(xtrain)
xtrain=scaler.transform(xtrain)
xdev=scaler.transform(xdev)

In [6]:
#用SVM训练模型 格点搜索 选取最优参数
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {"gamma":[1, 0.1, 0.01],"C":[0.1, 1, 10]}
grid_search = GridSearchCV(SVC(),param_grid,cv=5) 
grid_search.fit(xtrain,ytrain.ravel()) 
print("Test set score:{:.2f}".format(grid_search.score(xdev,ydev)))
print("Best parameters:{}".format(grid_search.best_params_))

Test set score:0.93
Best parameters:{'C': 10, 'gamma': 0.01}


In [180]:
#构建模型并对模型进行预测
from sklearn.metrics import accuracy_score
svc = SVC(C=10,gamma=0.01,class_weight='balanced')
svc.fit(xtrain,ytrain)
ypred = svc.predict(xdev)
print(accuracy_score(ydev,ypred))

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9523465703971119


In [181]:
#导入测试集预处理
test_data = sio.loadmat("data_test_final.mat")['data_test_final']
test_data = np.array(test_data, dtype=np.float64)
xtest = scaler.transform(test_data)

In [182]:
#生成测试集预测标签并导出csv文件进行保存
import pandas as pd
ytest = svc.predict(xtest)
data = pd.DataFrame(ytest)
data.to_csv("ytest.csv")